In [1]:
import configparser
import sys
import os
import json
from datetime import datetime
import re

from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.functions import col

from snowflake.ml.modeling.preprocessing import OneHotEncoder

# Create a ConfigParser object
config = configparser.ConfigParser()

# Define the path to the config file
config_path = os.path.expanduser("~/.snowsql/config") 
#Se comenta esta linea de codigo para usar el json con credenciales dentro del proyecto
#my_dir = os.path.dirname(os.path.abspath(__file__))
#config_path = os.path.join(my_dir, 'another_config.cfg')
#print(config_path)

# Read the contents of the file into the ConfigParser object
config.read(config_path)

# Access the values using the section and key
# Assuming the values you want are in the "connections.dev" section
dict_creds = {}

#Se comenta esta linea de codigo para usar el json con credenciales dentro del proyecto
dict_creds['account'] = config['connections.dev']['accountname']
dict_creds['user'] = config['connections.dev']['username']
dict_creds['password'] = config['connections.dev']['password']
dict_creds['role'] = config['connections.dev']['rolename']
dict_creds['database'] = config['connections.dev']['dbname']
dict_creds['warehouse'] = config['connections.dev']['warehousename']

# dict_creds['account'] = config['connections.dev']['account']
# dict_creds['user'] = config['connections.dev']['user']
# dict_creds['password'] = config['connections.dev']['password']
# dict_creds['role'] = config['connections.dev']['role']


# Setting up Snowpark Connection
session = Session.builder.configs(dict_creds).create()

In [7]:
from snowflake.ml.registry import Registry
reg = Registry(session=session, database_name="BANANA_QUALITY",  schema_name="DEV")
model_info = reg.show_models()
if model_info.empty:
    print("El DataFrame está vacío.")
else:
    print("El DataFrame no está vacío.")

El DataFrame está vacío.


In [6]:
print(model_info)

Empty DataFrame
Columns: []
Index: []


In [3]:
IMPORT_DIRECTORY_NAME = "snowflake_import_directory"
import_dir = sys._xoptions[IMPORT_DIRECTORY_NAME]
model_name = 'model.joblib.gz'

KeyError: 'snowflake_import_directory'

In [ ]:

session.file.get('@dev/model.joblib.gz', file:///tmp/model.joblib.gz)


In [9]:
import joblib
from tempfile import NamedTemporaryFile

#session.use_schema("DEV")


#with NamedTemporaryFile(delete=False, suffix='.joblib.gz') as tmp_file:
    # Descargar el archivo desde el stage al archivo temporal
session.file.get('@ML_MODELS/model.joblib.gz', "model.joblib.gz")
model_path = "model.joblib.gz"

# Cargar el modelo desde el archivo descargado
with open(model_path, 'rb') as f:
    model = joblib.load(f)

PermissionError: [Errno 13] Permission denied: 'model.joblib.gz'

In [15]:
# Función para cargar el modelo desde un internal stage
def load_model_from_stage(session: Session):
    # Crear un archivo temporal
    with NamedTemporaryFile(suffix='.joblib.gz') as tmp_file:
        # Descargar el archivo desde el stage al archivo temporal
        session.file.get('@ML_MODELS/model.joblib.gz', tmp_file.name)
        model_path = tmp_file.name

    # Cargar el modelo desde el archivo descargado
    with open(model_path, 'rb') as f:
        model = joblib.load(f)

    # Eliminar el archivo temporal
    os.remove(model_path)
    
    return model

In [16]:
model = load_model_from_stage(session)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\ERICKA~1.PER\\AppData\\Local\\Temp\\tmpcrisstpy.joblib.gz'

In [12]:
session.use_schema("DEV")
model = session.file.get('@ML_MODELS/model.joblib.gz', '/tmp')

In [15]:
%%!sudo chmod a+w /tmp

UsageError: Cell magic `%%!sudo` not found.


In [13]:
model_path = '/tmp/model.joblib.gz'
with open(model_path, 'rb') as f:
    model = joblib.load(f)

PermissionError: [Errno 13] Permission denied: '/tmp/model.joblib.gz'